In [7]:
import json
import torch
import pandas as pd

In [8]:
name = 'small_sample_train/000000coe.json'
with open(name) as f:
    data = json.load(f)

In [9]:
ep = data[0]
ep[0].keys()

dict_keys(['size', 'walls', 'fuse_walls', 'pin', 'key', 'objects', 'blocking', 'home', 'agent'])

For each episode, the only data that changes each frame is the agent data. And some fields are empty entirely.

In [36]:
print(ep[0]['size'])
print(ep[0]['objects']) #the goal object (with shape flag)
print(ep[0]['home']) #the home object where the agent starts (with shape of house) with rgb value 255, 105 180
print(ep[0]['agent'])

[200, 200]
[[[74, 60], 9, './shapes/train/objects/flag.png', [255, 255, 0]]]
[[47, 24], 6, './shapes/home.png', [255, 105, 180]]
[[47, 24], 6, './shapes/train/agents/kite.png', [0, 0, 128]]


In [16]:
for i in ep[0].keys():
    if ep[0][i] != ep[-1][i]:
        print(i, 'doesnt match \n')
for key in ep[0].keys():
    print(key, len(ep[0][key]))

agent doesnt match 

size 2
walls 50
fuse_walls 0
pin 0
key 0
objects 1
blocking 0
home 4
agent 4


In [24]:
print(len(ep)) #in a 129 frame episode, only 14 have anything changing
agent_changes = []
curr = ep[0]['agent']
for i,b in enumerate(ep):
    if b['agent'] != curr:
        print (i,b['agent'])
        curr = b['agent']
        agent_changes.append(b['agent'][0])
print(agent_changes)


129
31 [[48, 25], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
32 [[49, 27], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
33 [[51, 29], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
34 [[52, 31], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
35 [[54, 33], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
36 [[55, 35], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
37 [[57, 37], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
38 [[59, 39], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
39 [[60, 41], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
40 [[62, 43], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
41 [[63, 45], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
42 [[65, 47], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
43 [[66, 49], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
44 [[68, 51], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
45 [[69, 53], 6, './shapes/train/agents/kite.png', [0, 0, 128]]
[[48, 25], [49, 27], [51, 29], [52, 

In [ ]:
#if trying best to represent the data as an image, can make a 3d tensor to include a color channel that is 0 for
#all of the walls and 1 for the agent, goal, and home objects

In [52]:
(agent_change_indices).numpy()/20

array([[2.4 , 1.25],
       [2.45, 1.35],
       [2.55, 1.45],
       [2.6 , 1.55],
       [2.7 , 1.65],
       [2.75, 1.75],
       [2.85, 1.85],
       [2.95, 1.95],
       [3.  , 2.05],
       [3.1 , 2.15],
       [3.15, 2.25],
       [3.25, 2.35],
       [3.3 , 2.45],
       [3.4 , 2.55],
       [3.45, 2.65]])

In [44]:
#example of 3d color channel
torch.zeros(3,3,3)

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [46]:
agent_change_indices = torch.tensor(agent_changes)
agent_change_indices

tensor([[48, 25],
        [49, 27],
        [51, 29],
        [52, 31],
        [54, 33],
        [55, 35],
        [57, 37],
        [59, 39],
        [60, 41],
        [62, 43],
        [63, 45],
        [65, 47],
        [66, 49],
        [68, 51],
        [69, 53]])

In [60]:
object_tensor = torch.tensor(ep[0]['objects'][0][0])
object_tensor

tensor([74, 60])

In [ ]:
#walls are 1x1 blocks in a 10x10 grid scaled by factor of 20

In [13]:
walls = (pd.DataFrame([a for a,b in ep[0]['walls']],columns=['Wall_x','Wall_y'])/20).astype(int)  #honestly don't even need b if it's the same always (Make sure to check)
wall_x_indices = torch.tensor(walls['Wall_x'].values)
wall_y_indices = torch.tensor(walls['Wall_y'].values)

In [20]:
wall_grid = torch.zeros(10,10)
wall_grid[wall_x_indices,wall_y_indices] = 1
wall_grid

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1., 1., 0., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 1., 1.],
        [1., 0., 0., 0., 0., 1., 1., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [1., 0., 0., 0., 1., 0., 1., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
        [1., 0., 0., 0., 0., 0., 1., 0., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [63]:
#naive representation of data for prediction:
data = wall_grid.flatten()
data = torch.cat((data,object_tensor/200,agent_change_indices[0]/200),0)

In [64]:
data

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.3700, 0.3000, 0.2400, 0.1250])

In [74]:
def get_agent_change_indices(ep):
    agent_changes = []
    curr = ep[0]['agent']
    for i,b in enumerate(ep):
        if b['agent'] != curr:
            curr = b['agent']
            agent_changes.append(b['agent'][0])
    return torch.tensor(agent_changes)


In [73]:
def naive_preprocess(episode):
    static_data = episode[0]
    #prob a more efficient way to process walls w/o dataframe or pandas
    walls = (pd.DataFrame([a for a,b in static_data['walls']],columns=['Wall_x','Wall_y'])/20).astype(int) 
    wall_x_indices = torch.tensor(walls['Wall_x'].values)
    wall_y_indices = torch.tensor(walls['Wall_y'].values)
    wall_grid = torch.zeros(10,10)
    wall_grid[wall_x_indices,wall_y_indices] = 1
    agent_change_indices = get_agent_change_indices(episode)
    object_tensor = torch.tensor(static_data['objects'][0][0])
    data = wall_grid.flatten()
    data = torch.cat((data,object_tensor/200,agent_change_indices[0]/200),0)
    return data
naive_preprocess(ep)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 1.0000, 0.0000, 1.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 0.3700, 0.3000, 0.2400, 0.1250])